In [46]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
import random

In [47]:
data=pd.read_csv("./data/PremierLeague/allAtt_onehot_large_train.csv")
dataT=pd.read_csv("./data/PremierLeague/allAtt_onehot_large_test.csv")
print(data.head())
print(data.shape)

FTR  HTGS  ATGS  HTGC  ATGC  HTP  ATP  HM1  HM2  HM3  ...  ATWinStreak5  \
0    0   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
1    0   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
2    1   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
3    0   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   
4    1   0.0   0.0   0.0   0.0  0.0  0.0    2    2    2  ...             0   

   ATLossStreak3  ATLossStreak5  HTGD  ATGD  DiffPts  DiffFormPts  DiffLP  \
0              0              0   0.0   0.0      0.0          0.0   -11.0   
1              0              0   0.0   0.0      0.0          0.0     8.0   
2              0              0   0.0   0.0      0.0          0.0     6.0   
3              0              0   0.0   0.0      0.0          0.0    -9.0   
4              0              0   0.0   0.0      0.0          0.0    -8.0   

   final1  final2  
0     1.0     0.0  
1     1.0     0.0  
2     0.0  

In [48]:
#data = data[['HTGS','ATGS','HTP','ATP','DiffLP','final1']]
#dataT = dataT[['HTGS','ATGS','HTP','ATP','DiffLP','final1']]
data = data[['HTGS','ATGS','HTP','ATP','HM1','AM1', 'DiffLP','final1']]
dataT = dataT[['HTGS','ATGS','HTP','ATP','HM1','AM1','DiffLP','final1']]

In [49]:
data.head()

,HTGS,ATGS,HTP,ATP,HM1,AM1,DiffLP,final1
0,0.0,0.0,0.0,0.0,2,2,-11.0,1.0
1,0.0,0.0,0.0,0.0,2,2,8.0,1.0
2,0.0,0.0,0.0,0.0,2,2,6.0,0.0
3,0.0,0.0,0.0,0.0,2,2,-9.0,1.0
4,0.0,0.0,0.0,0.0,2,2,-8.0,0.0


In [50]:
data = data.sort_values('HTGS')
dataT = dataT.sort_values('HTGS')

In [51]:
x = np.array(data)
x_t = np.array(dataT)

In [52]:
print(x.shape)
print(x_t.shape)

(2620, 8)
(800, 8)


In [53]:
x_train = np.array(x[:, :7])
x_test = np.array(x_t[:, :7])
y_train = np.array(x[:, 7])
y_test = np.array(x_t[:, 7])

In [54]:
from qore_sdk.client import WebQoreClient
from qore_sdk.utils import sliding_window

username = '*****'
password = '*****'
endpoint = '*****'

client = WebQoreClient(username, password, endpoint=endpoint)

In [55]:
X, y= sliding_window(x_train, 10, 4, axis=0, y=y_train,y_def='mode', y_axis=0)

In [56]:
print(X.shape)
print(y.shape)

(653, 10, 7)
(653, 1)


In [57]:
client.classifier_train(X, y)

{'res': 'ok', 'train_time': 0.8582723140716553}

In [58]:
X_test, y_test = sliding_window(x_test, 10, 5, axis=0, y=y_test,y_def='mode', y_axis=0)

In [59]:
print(X_test.shape)
print(y_test.shape)

(159, 10, 7)
(159, 1)


In [60]:
res = client.classifier_predict(X_test)
report = classification_report(y_test, res['Y'])
print(report)

precision    recall  f1-score   support

         0.0       0.73      0.90      0.81       104
         1.0       0.68      0.38      0.49        55

    accuracy                           0.72       159
   macro avg       0.71      0.64      0.65       159
weighted avg       0.71      0.72      0.70       159

